<a href="https://colab.research.google.com/github/LGdatacruncher/DataScienceHub/blob/main/Deploying_a_Keras_Model_via_Flask_and_Ngrok_for_Digit_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Objective:

To build a Flask web application that serves a Keras model trained on the MNIST dataset, making predictions on handwritten digit images. The application is accessible via a public URL using Ngrok, enabling easy demonstration and testing of the model from any location.**

In [ ]:
# imports

import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K


In [ ]:
# Hyperparameters

num_classes = 10
batch_size = 128
epochs = 12


In [ ]:
# Image Resolution

img_rows, img_cols = 28, 28

# Loading the data.

(x_train, y_train), (x_test, y_test) = mnist.load_data()


In [ ]:
# Preparing the data

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)


x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)



x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


In [ ]:
# Creating the Model

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),activation='relu',input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])



/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
# Training the Model

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))



Epoch 1/12
469/469 ━━━━━━━━━━━━━━━━━━━━ 14s 16ms/step - accuracy: 0.1121 - loss: 2.3059 - val_accuracy: 0.2147 - val_loss: 2.2670
Epoch 2/12
469/469 ━━━━━━━━━━━━━━━━━━━━ 4s 8ms/step - accuracy: 0.1993 - loss: 2.2623 - val_accuracy: 0.3908 - val_loss: 2.2154
Epoch 3/12
469/469 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.2973 - loss: 2.2119 - val_accuracy: 0.4848 - val_loss: 2.1510
Epoch 4/12
469/469 ━━━━━━━━━━━━━━━━━━━━ 5s 7ms/step - accuracy: 0.3720 - loss: 2.1492 - val_accuracy: 0.5482 - val_loss: 2.0667
Epoch 5/12
469/469 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - accuracy: 0.4343 - loss: 2.0672 - val_accuracy: 0.6016 - val_loss: 1.9562
Epoch 6/12
469/469 ━━━━━━━━━━━━━━━━━━━━ 5s 7ms/step - accuracy: 0.4799 - loss: 1.9640 - val_accuracy: 0.6465 - val_loss: 1.8158
Epoch 7/12
469/469 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.5222 - loss: 1.8311 - val_accuracy: 0.6902 - val_loss: 1.6469
Epoch 8/12
469/469 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - accuracy: 0.5551 - loss: 1.6885 - val_accuracy: 

In [ ]:
# Evaluating the Predictions on the Model

score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.8957334160804749
Test accuracy: 0.8058000206947327


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
import os

# Load and preprocess the MNIST dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train / 255.0
x_test = x_test / 255.0
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

# Build the model
model = Sequential([
    Flatten(input_shape=(28, 28)),
    Dense(128, activation='relu'),
    Dense(10, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
model.fit(x_train, y_train, epochs=5, batch_size=32, validation_split=0.2)

# Save the model architecture to JSON file
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)

# Save the model weights to HDF5 file with the correct extension
model.save_weights("model_weights.weights.h5")  # Use .weights.h5 extension

# Ensure that model_weights.weights.h5 exists before renaming (if needed)
if os.path.exists("model_weights.h5"):
    os.rename("model_weights.h5", "model_weights.weights.h5")
else:
    print("Error: 'model_weights.h5' does not exist.")

# Load JSON file that contains the model architecture
with open('model.json', 'r') as json_file:
    loaded_model_json = json_file.read()

# Use model_from_json to load the model architecture
loaded_model = model_from_json(loaded_model_json)

# Load weights into the model
loaded_model.load_weights("model_weights.weights.h5")  # Ensure the filename matches
print("Model loaded successfully.")

# Compile the loaded model (if needed for evaluation or prediction)
loaded_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


Epoch 1/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.8640 - loss: 0.4854 - val_accuracy: 0.9528 - val_loss: 0.1579
Epoch 2/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9609 - loss: 0.1368 - val_accuracy: 0.9671 - val_loss: 0.1097
Epoch 3/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9743 - loss: 0.0883 - val_accuracy: 0.9658 - val_loss: 0.1133
Epoch 4/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step - accuracy: 0.9819 - loss: 0.0627 - val_accuracy: 0.9733 - val_loss: 0.0922
Epoch 5/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9850 - loss: 0.0491 - val_accuracy: 0.9752 - val_loss: 0.0854
Error: 'model_weights.h5' does not exist.
Model loaded successfully.


In [ ]:
# Save the model architecture to JSON file
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)

# Save the model weights to HDF5 file with the correct extension
model.save_weights("model_weights.weights.h5")  # Use .weights.h5 extension
print("Model weights saved successfully.")

Model weights saved successfully.


In [ ]:
from tensorflow.keras.models import model_from_json

# Load JSON file that contains the model architecture
with open('model.json', 'r') as json_file:
    loaded_model_json = json_file.read()

# Use model_from_json to load the model architecture
loaded_model = model_from_json(loaded_model_json)

# Load weights into the model
loaded_model.load_weights("model_weights.weights.h5")  # Ensure the filename matches
print("Model loaded successfully.")

# Compile the loaded model (if needed for evaluation or prediction)
loaded_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


Model loaded successfully.


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import model_from_json

# Load JSON file that contains the model architecture
with open('model.json', 'r') as json_file:
    loaded_model_json = json_file.read()

# Use model_from_json to load the model architecture
loaded_model = model_from_json(loaded_model_json)

# Load weights into the model
loaded_model.load_weights("model_weights.weights.h5")  # Correct filename for the weights
print("Loaded Model from disk")

# Compile the loaded model (if needed for evaluation or prediction)
loaded_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


Loaded Model from disk


In [ ]:
!pip install pyngrok

In [ ]:
!pip install flask_ngrok

In [ ]:
!curl -s https://ngrok-agent.s3.amazonaws.com/ngrok.asc | sudo tee /etc/apt/trusted.gpg.d/ngrok.asc >/dev/null && echo "deb https://ngrok-agent.s3.amazonaws.com buster main" | sudo tee /etc/apt/sources.list.d/ngrok.list && sudo apt update && sudo apt install ngrok

deb https://ngrok-agent.s3.amazonaws.com buster main
Get:1 https://ngrok-agent.s3.amazonaws.com buster InRelease [20.3 kB]
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:8 https://ngrok-agent.s3.amazonaws.com buster/main amd64 Packages [4,888 B]
Hit:9 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 http://security.

In [ ]:
!ls

model.h5  model.json  sample_data


In [ ]:
!ngrok authtoken 2idqNS37zRV2XDajxEFWqqu5LMf_6om6saR32Nec664Ys8S4k

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
!ngrok --version


ngrok version 3.12.0


In [ ]:
from flask import Flask
from pyngrok import ngrok

app = Flask(__name__)

@app.route("/")
def home():
    return "Hello, Flask with Ngrok!"

if __name__ == "__main__":
    # Open a ngrok tunnel to the HTTP server
    public_url = ngrok.connect(5000)
    print(" * ngrok tunnel URL:", public_url)

    # Run the Flask app
    app.run()


 * ngrok tunnel URL: NgrokTunnel: "https://86b7-34-145-121-22.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [01/Jul/2024 14:46:40] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [01/Jul/2024 14:46:41] "GET /favicon.ico HTTP/1.1" 404 -


In [ ]:
# Terminate the ngrok tunnel
ngrok.kill()


In [ ]:
# Save the model again (if you have the training script or can re-train)
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten

# Example model for demonstration
model = Sequential([
    Flatten(input_shape=(28, 28)),
    Dense(128, activation='relu'),
    Dense(10, activation='softmax')
])

# Compile and save the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
# Train and save
# model.fit(...)
model.save('model.h5')


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
import tensorflow as tf

# Path to your model file
model_path = 'model.h5'

try:
    model = tf.keras.models.load_model(model_path)
    print("Model loaded successfully.")
except Exception as e:
    print(f"Error loading model: {e}")


Model loaded successfully.


In [ ]:
# Continue training if necessary
loaded_model.fit(x_train, y_train, epochs=5, batch_size=32, validation_split=0.2)


NameError: name 'loaded_model' is not defined

In [ ]:
# Evaluate the model
loss, accuracy = loaded_model.evaluate(x_test, y_test)
print(f"Test loss: {loss}")
print(f"Test accuracy: {accuracy}")


313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9746 - loss: 0.0920
Test loss: 0.08193515241146088
Test accuracy: 0.9776999950408936


In [ ]:
# Make predictions
sample_image = x_test[0].reshape(1, 28, 28)  # Reshape as a single sample
prediction = loaded_model.predict(sample_image)
print("Prediction:", prediction)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 183ms/step
Prediction: [[1.6032643e-10 1.9806288e-13 1.4874885e-07 2.8505469e-06 1.4659546e-18
  8.0653895e-12 3.8889173e-18 9.9999702e-01 4.1181669e-11 6.9716815e-09]]


In [ ]:
from flask import Flask, render_template, request
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
from PIL import Image
import base64
from io import BytesIO
from pyngrok import ngrok

# Initialize Flask application
app = Flask(__name__)

# Load the Keras model
model_path = 'model.h5'  # Path to your trained model file
try:
    model = load_model(model_path)
    print("Model loaded successfully.")
except Exception as e:
    print(f"Error loading model: {e}")

@app.route('/')
def index_view():
    return render_template('index.html')

def convertImage(imgData1):
    imgstr = base64.b64decode(imgData1.split(',')[1])
    img = Image.open(BytesIO(imgstr)).convert('L')
    img = img.resize((28, 28))
    img = np.array(img)
    img = np.invert(img)
    img = img.reshape(1, 28, 28, 1)
    return img

@app.route('/predict/', methods=['POST'])
def predict():
    imgData = request.get_data(as_text=True)
    img = convertImage(imgData)
    predictions = model.predict(img)
    predicted_class = np.argmax(predictions, axis=1)
    return str(predicted_class[0])

if __name__ == '__main__':
    # Open a ngrok tunnel to the HTTP server
    public_url = ngrok.connect(5000)
    print(" * ngrok tunnel URL:", public_url)

    # Run the Flask app
    app.run(debug=True, port=5000)


Error loading model: No file or directory found at model.h5


ERROR:pyngrok.process.ngrok:t=2024-07-01T14:44:18+0000 lvl=eror msg="failed to reconnect session" obj=tunnels.session err="authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n"
ERROR:pyngrok.process.ngrok:t=2024-07-01T14:44:18+0000 lvl=eror msg="session closing" obj=tunnels.session err="authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n"
ERROR:pyngrok.process.ngrok:t=2024-07-01T14:44:18+0000 lvl=eror msg="terminating with error" obj=app err="authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your aut

PyngrokNgrokError: The ngrok process errored on start: authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n.